# Data Wrangling with Spark SQL Quiz

This quiz uses the same dataset and most of the same questions from the earlier "Quiz - Data Wrangling with Data Frames Jupyter Notebook." For this quiz, however, use Spark SQL instead of Spark Data Frames.

In [2]:
from pyspark.sql import SparkSession

# TODOS: 
# 1) import any other libraries you might need
# 2) instantiate a Spark session 
# 3) read in the data set located at the path "data/sparkify_log_small.json"
# 4) create a view to use with your SQL queries
# 5) write code to answer the quiz questions 

spark = SparkSession\
        .builder\
        .appName("Data wrangling with Spark SQL")\
        .getOrCreate()

path = "data/sparkify_log_small.json"
user_log = spark.read.json(path)
user_log.createOrReplaceTempView("user_log_table")
user_log.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



# Question 1

Which page did user id ""(empty string) NOT visit?

In [9]:
# TODO: write your code to answer question 1
spark.sql("""SELECT DISTINCT page 
             FROM user_log_table
             WHERE page NOT IN (SELECT DISTINCT page 
                                FROM user_log_table
                                WHERE userId = '')
""").show()


+----------------+
|            page|
+----------------+
|Submit Downgrade|
|       Downgrade|
|          Logout|
|   Save Settings|
|        Settings|
|        NextSong|
|         Upgrade|
|           Error|
|  Submit Upgrade|
+----------------+



# Question 2 - Reflect

Why might you prefer to use SQL over data frames? Why might you prefer data frames over SQL?

# Question 3

How many female users do we have in the data set?

In [4]:
# TODO: write your code to answer question 3
spark.sql("""
    SELECT gender, count(gender)
    FROM (SELECT DISTINCT userId, gender
          FROM user_log_table)
    GROUP BY gender
""").show()

+------+-------------+
|gender|count(gender)|
+------+-------------+
|     F|          462|
|  null|            0|
|     M|          501|
+------+-------------+



# Question 4

How many songs were played from the most played artist?

In [ ]:
# TODO: write your code to answer question 4

# Question 5 (challenge)

How many songs do users listen to on average between visiting our home page? Please round your answer to the closest integer.

In [ ]:
# TODO: write your code to answer question 5